In [20]:
import torch 
from torch import nn, optim
from torch.utils.data import DataLoader 

from torchvision import datasets, transforms

import numpy as np 
import matplotlib.pyplot as plt
import PIL

%matplotlib inline


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"DEVICE {device}")

DEVICE cuda


In [8]:
train_transform = transforms.ToTensor()


train_data = datasets.MNIST(root='../data/', download=True, train=True,  transform=train_transform)
train_dataloader = DataLoader(train_data, 32)

# 1, 28, 28


100%|██████████| 9912422/9912422 [00:00<00:00, 11203736.70it/s]


Extracting ../data/MNIST\raw\train-images-idx3-ubyte.gz to ../data/MNIST\raw



100%|██████████| 28881/28881 [00:00<00:00, 21493203.30it/s]

Extracting ../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../data/MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 11526850.68it/s]


Extracting ../data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../data/MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data/MNIST\raw



In [14]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10) # 10 class
        )
        
    
    def forward(self, x):
        return self.model(x)

In [16]:
clf = ImageClassifier().to(device)

opt = optim.Adam(clf.parameters(), lr=1e-3)

loss_fn = nn.CrossEntropyLoss()

In [18]:
for epoch in range(10):
    for batch in train_dataloader:
        X, y = batch 
        X, y = X.to(device), y.to(device)
        y_pred = clf(X)
        loss = loss_fn(y_pred, y)
        
        # apply backprop
        opt.zero_grad()
        loss.backward()
        opt.step()
        
    print(f"Epoch : {epoch} loss is {loss.item()}")
    

with open('model_state.pt', 'wb') as f: 
    torch.save(clf.state_dict(), f)
    
        

Epoch : 0 loss is 0.019827350974082947
Epoch : 1 loss is 0.0022395155392587185
Epoch : 2 loss is 0.0008888730662874877
Epoch : 3 loss is 0.000284913694486022
Epoch : 4 loss is 0.0001725824986351654
Epoch : 5 loss is 1.2512457033153623e-05
Epoch : 6 loss is 1.7797518012230285e-05
Epoch : 7 loss is 9.201435773320554e-07
Epoch : 8 loss is 2.9057196115900297e-07
Epoch : 9 loss is 5.625161634270626e-07


In [19]:
with open('model_state.pt', 'rb') as f:
    clf.load_state_dict(torch.load(f))

In [36]:
img = PIL.Image.open('../data/MNIST/number_9.jpg')
img_tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)


print(torch.argmax(clf(img_tensor)))


tensor(9, device='cuda:0')
